In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
predictions_csv_path = r"training data\predictions.csv"

predictions_array_df = pd.read_csv(predictions_csv_path)

In [8]:
def current(B, arrJ, arrC, y):
    curr = 0
    limit = int(len(arrJ) / 2)
    for n in range(limit):
        numOfSegments = 5 if (arrJ[2 * n + 1] - arrJ[2 * n]) < 0.05 else int(100 * (arrJ[2 * n + 1] - arrJ[2 * n]))
        sizeOfSegment = float((arrJ[2 * n + 1] - arrJ[2 * n]) / numOfSegments)
        for i in range(numOfSegments):
            curr += arrC[n] * np.sin(y + (2 * np.pi * B) * (arrJ[2 * n] + i * sizeOfSegment)) * (1/numOfSegments)
    return curr

def criticalCurrent(density, arrJ):
    criticalCurrents = []
    junctionWidths = []
    for i in range(len(arrJ)//2):
        junctionWidths.append(arrJ[2*i+1] - arrJ[2*i])
    for i in range (len(junctionWidths)):
        criticalCurrents.append(junctionWidths[i] * density[i])
    return criticalCurrents

def maxCurrent(B, arrayJ, arrayC):
    Y=np.linspace(0, 2*np.pi, 150)
    dummyArray=[]
    for gamma in Y:
        dummyArray.append(current(B, arrayJ, arrayC, gamma))
    return max(dummyArray)

def IMaxPoint(arrayOfJunctions):
    I_c = []
    criticalCurrents = criticalCurrent(np.ones(len(arrayOfJunctions)//2), arrayOfJunctions)
    exp_csv_path = r"experimental data\experimental csvs\5.0uV_normalized.csv"
    exp_arr = pd.read_csv(exp_csv_path).values
    FluxField = [pair[0] for pair in exp_arr]
    for f in FluxField:
        I_c.append(maxCurrent(B=f, arrayJ=arrayOfJunctions, arrayC=criticalCurrents) / np.sum(criticalCurrents))
    return I_c

def cost_function(basis_data, generated_data):
    diff = basis_data - generated_data
    for i in range(len(diff)):
        diff[i] = diff[i]**2
    return np.sum(diff)

def process_dataframe(df):
    # Sort each row
    df = df.apply(sorted, axis=1, result_type='expand')
    
    # Subtract the smallest value in each row
    df = df.subtract(df.min(axis=1), axis=0)
    
    # Normalize by the greatest value in each row
    df = df.divide(df.max(axis=1), axis=0)
    
    return df

In [9]:
I_c = []

exp_data = pd.read_csv(r"experimental data\experimental csvs\5.0uV_normalized.csv")

exp_data = exp_data.drop(exp_data.columns[0], axis=1)

exp_data = exp_data.values

predictions_array_df = process_dataframe(predictions_array_df)

predictions = predictions_array_df.values

array_of_costs = []

exp_data

# predictions

array([[0.10234627],
       [0.10430817],
       [0.10673394],
       ...,
       [0.08508925],
       [0.08070811],
       [0.07953125]])

In [ ]:
for i in range(len(predictions)//10):
    I_c = IMaxPoint(arrayOfJunctions=predictions[10 * i])
    array_of_costs.append(cost_function(I_c, exp_data))

In [ ]:
plt.plot(np.linspace(0, len(array_of_costs), array_of_costs), array_of_costs)
plt.grid()
plt.show() 